# 2CS-SIL2/SIQ2 Lab04. Naïve Bayes

<p style='text-align: right;font-style: italic;'>Designed by: Mr. Abdelkrime Aries</p>

In this lab, we will learn about Naive Bayes by testing 2 implementations:
- Multinomial Naïve Bayes
- Gaussian Naïve Bayes

**Team:**
- **Member 01**: OUIKENE Youcef
- **Member 02**: AOULMI Lina
- **Group**: SIQ2

In [1]:
import sys, timeit
from typing          import Tuple, List, Type
from collections.abc import Callable

sys.version

'3.11.12 (main, Apr  9 2025, 08:55:54) [GCC 11.4.0]'

In [2]:
import numpy             as np
import pandas            as pd
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

np.__version__, pd.__version__, matplotlib.__version__

('2.0.2', '2.2.2', '3.10.0')

In [3]:
import sklearn

from sklearn.naive_bayes   import CategoricalNB
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics       import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection         import train_test_split
from sklearn.naive_bayes             import MultinomialNB, GaussianNB
from sklearn.linear_model            import LogisticRegression
from sklearn.tree                    import DecisionTreeClassifier
from sklearn.metrics                 import precision_score, recall_score
import timeit


sklearn.__version__

'1.6.1'

## I. Algorithms implementation

In this section, we will try to implement multinomial Naive Bayes.


**>> Try to use "numpy" which will save a lot of time and effort**

In [4]:
# Dataset play

# outlook & temperature & humidity & windy
Xplay = np.array([
    ['sunny'   , 'hot' , 'high'  , 'no'],
    ['sunny'   , 'hot' , 'high'  , 'yes'],
    ['overcast', 'hot' , 'high'  , 'no'],
    ['rainy'   , 'mild', 'high'  , 'no'],
    ['rainy'   , 'cool', 'normal', 'no'],
    ['rainy'   , 'cool', 'normal', 'yes'],
    ['overcast', 'cool', 'normal', 'yes'],
    ['sunny'   , 'mild', 'high'  , 'no'],
    ['sunny'   , 'cool', 'normal', 'no'],
    ['rainy'   , 'mild', 'normal', 'no'],
    ['sunny'   , 'mild', 'normal', 'yes'],
    ['overcast', 'mild', 'high'  , 'yes'],
    ['overcast', 'hot' , 'normal', 'no'],
    ['rainy'   , 'mild', 'high'  , 'yes']
])

Yplay = np.array([
    'no',
    'no',
    'yes',
    'yes',
    'yes',
    'no',
    'yes',
    'no',
    'yes',
    'yes',
    'yes',
    'yes',
    'yes',
    'no'
])

len(Xplay), len(Yplay)

(14, 14)

In [5]:
# height & weight & footsize & person
Xperson = np.array([
    [182., 81.6, 30.],
    [180., 86.2, 28.],
    [170., 77.1, 30.],
    [180., 74.8, 25.],
    [152., 45.4, 15.],
    [168., 68.0, 20.],
    [165., 59.0, 18.],
    [175., 68.0, 23.]
])

Yperson = np.array([
    'male', 'male', 'male', 'male',
    'female', 'female', 'female', 'female'
])

len(Xperson), len(Yperson)

(8, 8)

### I.1. Prior statistics

Given an output list $Y[M]$, the probability of each class $c$ is estimated as:
$$p(c) = \frac{\#(Y = c)}{|Y|}$$

In here, we want to store the frequencies of different classes.
Our function must return two lists:
- One containing the names of unique classes.
- Another containing their frequencies.

In [6]:
# TODO: Prior statistics
def fit_prior(Y: 'np.ndarray[M](str)') -> Tuple['np.ndarray[K](str)', 'np.ndarray[K](int)']:
    return np.unique(Y,return_counts=True)

#=====================================================================
# UNIT TEST
#=====================================================================
# Result:
# ((array(['no', 'yes'], dtype='<U3'), array([5, 9])),
#  (array(['female', 'male'], dtype='<U6'), array([4, 4])))
#---------------------------------------------------------------------

fit_prior(Yplay), fit_prior(Yperson)

((array(['no', 'yes'], dtype='<U3'), array([5, 9])),
 (array(['female', 'male'], dtype='<U6'), array([4, 4])))

### I.2. Multinomial Law

In this section, we will implement multinomial naive Bayes from scratch using Numpy.

#### I.2.1. Multinomial Likelihood statistics

Given:
- $A$: a categorical feature
- $Y$: the ouput
- $C$: the classes

The function takes as argument $A, Y, C$ previously described.
It must return:
- $V$: unique values of this feature (feature's categories)
- $S[|C|, |V|]$: a matrix containing count $\#(Y = c \wedge A = v),\, \forall c \in C, \forall v \in A$

In [7]:
# TODO: Multinomial Likelihood statistics
def fit_multinomial_likelihood(A: 'np.ndarray[M](str)',
                               Y: 'np.ndarray[M](str)',
                               C: 'np.ndarray[C](str)'
                               ) -> Tuple['np.ndarray[V](str)', 'np.ndarray[C, V](int)']:
    V = np.unique(A)
    S = np.zeros([len(C),len(V)],dtype=int)
    for i, c in enumerate(C):
        for j ,v in enumerate(V):
            S[i,j]=np.sum((A==v)&(Y==c))
    return V,S

#=====================================================================
# UNIT TEST
#=====================================================================
# Result:
# ((array(['overcast', 'rainy', 'sunny'], dtype='<U8'),
#   array([[0, 2, 3],
#          [4, 3, 2]])),
#  (array(['cool', 'hot', 'mild'], dtype='<U8'),
#   array([[1, 2, 2],
#          [3, 2, 4]])))
#---------------------------------------------------------------------
C_t = np.array(['no', 'yes'])
fit_multinomial_likelihood(Xplay[:, 0], Yplay, C_t), fit_multinomial_likelihood(Xplay[:, 1], Yplay, C_t)

((array(['overcast', 'rainy', 'sunny'], dtype='<U8'),
  array([[0, 2, 3],
         [4, 3, 2]])),
 (array(['cool', 'hot', 'mild'], dtype='<U8'),
  array([[1, 2, 2],
         [3, 2, 4]])))

#### I.2.2. Multinomial Likelihood training

**Nothing to code here, although you have to know how it functions for next use**

This function aims to generate parameters $\theta$.
In our case, paramters are diffrent from those of *logistic regrssion*.
They are a dictionary (map) with two entries:
- "prior": a dictionary having "vocab" a list of values and "freq" a list of their respective frequencies.
- "likelihood": a list of dictionaries representing statistics of each feature (the same order of $X$ features)

In [8]:
def fit_multinomial_NB(X: 'np.ndarray[M, N](str)',
                       Y: 'np.ndarray[M](str)'
                       ) -> object:

    Theta   = {'prior': {}, 'likelihood': []}

    Theta['prior']['vocab'], Theta['prior']['freq'] = fit_prior(Y)

    for j in range(X.shape[1]):
        likelihood = {}
        likelihood['vocab'], likelihood['freq'] = fit_multinomial_likelihood(X[:, j], Y, Theta['prior']['vocab'])
        Theta['likelihood'].append(likelihood)

    return Theta


#=====================================================================
# UNIT TEST
#=====================================================================
# Result:
# {'prior': {'vocab': array(['no', 'yes'], dtype='<U3'), 'freq': array([5, 9])},
#  'likelihood': [{'vocab': array(['overcast', 'rainy', 'sunny'], dtype='<U8'),
#    'freq': array([[0, 2, 3],
#           [4, 3, 2]])},
#   {'vocab': array(['cool', 'hot', 'mild'], dtype='<U8'),
#    'freq': array([[1, 2, 2],
#           [3, 2, 4]])},
#   {'vocab': array(['high', 'normal'], dtype='<U8'),
#    'freq': array([[4, 1],
#           [3, 6]])},
#   {'vocab': array(['no', 'yes'], dtype='<U8'),
#    'freq': array([[2, 3],
#           [6, 3]])}]}
#---------------------------------------------------------------------
Theta_play = fit_multinomial_NB(Xplay, Yplay)

Theta_play

{'prior': {'vocab': array(['no', 'yes'], dtype='<U3'), 'freq': array([5, 9])},
 'likelihood': [{'vocab': array(['overcast', 'rainy', 'sunny'], dtype='<U8'),
   'freq': array([[0, 2, 3],
          [4, 3, 2]])},
  {'vocab': array(['cool', 'hot', 'mild'], dtype='<U8'),
   'freq': array([[1, 2, 2],
          [3, 2, 4]])},
  {'vocab': array(['high', 'normal'], dtype='<U8'),
   'freq': array([[4, 1],
          [3, 6]])},
  {'vocab': array(['no', 'yes'], dtype='<U8'),
   'freq': array([[2, 3],
          [6, 3]])}]}

#### I.2.3. Multinomial Likelihood prediction

Given:
- $A$: a categorical feature
- $V$: unique values of this feature (feature's categories)
- $Y$: the ouput
- $C$: the classes
- $\alpha$: smoothing factor

Log likelihood is calculated as:
$$ \log p(A=v|Y=c) = \log(\#(Y = k \wedge A = v) + \alpha) - \log(\#(y = k) + \alpha * |V|)$$


In [9]:
# You can use this function in the next implimentation
# It takes a list of unique values V and a given value v
# It returns the position of v in V
# If v does not exist in V, it rturns -1
def find_idx(V: np.ndarray, v: str) -> int:
    k = np.argwhere(V == v).flatten()
    if len(k):
        return k[0]
    return -1

V_t = np.array(['One', 'Two', 'Three'])
find_idx(V_t, 'Two'), find_idx(V_t, 'Four')

(np.int64(1), -1)

In [10]:
# TODO: Multinomial Likelihood prediction
def predict_multinomial_NB1(v: str,
                            j: int,
                            Theta: object,
                            alpha: float = 0.
                            ) -> 'np.ndarray[C](float)':

    k = find_idx(Theta['likelihood'][j]['vocab'], v)

    if (k == -1):

        return np.log(alpha) - np.log(Theta['prior']['freq'] + alpha * len(Theta['likelihood'][j]['vocab']))

    else:

        return np.log(Theta['likelihood'][j]['freq'][:, k] + alpha) - np.log(Theta['prior']['freq'] + alpha * len(Theta['likelihood'][j]['vocab']))

#=====================================================================
# UNIT TEST
#=====================================================================
# Result:
# (array([-0.91629073, -1.09861229]), array([-2.07944154, -2.48490665]))
#---------------------------------------------------------------------

X_t = np.array([
    ['rainy', 'cool', 'normal', 'yes'],
    ['snowy', 'cool', 'normal', 'yes'],
    ['sunny', 'hot' , 'normal', 'no']
])

predict_multinomial_NB1('rainy', 0, Theta_play, alpha=0.), \
    predict_multinomial_NB1('snowy', 0, Theta_play, alpha=1.)

(array([-0.91629073, -1.09861229]), array([-2.07944154, -2.48490665]))

### I.3. Normal (Gaussian) Law

In this section, we will implement gaussian naive Bayes from scratch using Numpy.

#### I.3.1. Gaussian Likelihood statistics

Given:
- $A$: a categorical feature
- $Y$: the ouput
- $C$: the classes

The function takes as argument $A, Y, C$ previously described.
It must return $S[|C|, 2, N]$; a tensor having these dimensions:
- first dimension: each element represents one class's statistics
- second dimension: 1st element represents means; 2ns element represents variances
- third dimension: each element represents mean/variance of the respective feature

In [11]:
# TODO: Gaussian Likelihood statistics
def fit_gaussian_likelihood(X: 'np.ndarray[M](float)',
                            Y: 'np.ndarray[M](str)',
                            C: 'np.ndarray[C](str)'
                            ) -> Tuple['np.ndarray[C, 2, N](float)']:
    XY = np.column_stack([X,Y])
    S = np.zeros([len(C),2,X.shape[1]])
    for i in range(len(C)):
        for j in range(X.shape[1]):
            # taking either male occurences or female occurences
            cX = XY[XY[:, -1] == C[i], j].astype(float)
            S[i,0,j]=np.mean(cX)
            S[i,1,j]=np.var(cX,ddof=1) # divide by n-1 and not n, so use ddof = 1
    return S

#=====================================================================
# UNIT TEST
#=====================================================================
# Result:
# array([[[165.        ,  60.1       ,  19.        ],
#         [ 92.66666667, 114.04      ,  11.33333333]],

#        [[178.        ,  79.925     ,  28.25      ],
#         [ 29.33333333,  25.47583333,   5.58333333]]])
#---------------------------------------------------------------------
C_t = np.array(['female', 'male'])
fit_gaussian_likelihood(Xperson, Yperson, C_t)

array([[[165.        ,  60.1       ,  19.        ],
        [ 92.66666667, 114.04      ,  11.33333333]],

       [[178.        ,  79.925     ,  28.25      ],
        [ 29.33333333,  25.47583333,   5.58333333]]])

#### I.3.2. Gaussian Likelihood training

**Nothing to code here, although you have to know how it functions for next use**

This function aims to generate parameters $\theta$.
In our case, paramters are diffrent from those of *logistic regrssion*.
They are a dictionary (map) with two entries:
- "prior": a dictionary having "vocab" a list of values and "freq" a list of their respective frequencies.
- "likelihood": a tensor of shape $[|C|, 2, N]$ containing likelihood statistics

In [12]:
def fit_gaussian_NB(X: 'np.ndarray[M, N](str)',
                    Y: 'np.ndarray[M](str)'
                    ) -> object:

    Theta   = {'prior': {}, 'likelihood': []}

    Theta['prior']['vocab'], Theta['prior']['freq'] = fit_prior(Y)
    Theta['likelihood'] = fit_gaussian_likelihood(X, Y, Theta['prior']['vocab'])

    return Theta


#=====================================================================
# UNIT TEST
#=====================================================================
# Result:
# {'prior': {'vocab': array(['female', 'male'], dtype='<U6'),
#   'freq': array([4, 4])},
#  'likelihood': array([[[165.        ,  60.1       ,  19.        ],
#          [ 92.66666667, 114.04      ,  11.33333333]],

#         [[178.        ,  79.925     ,  28.25      ],
#          [ 29.33333333,  25.47583333,   5.58333333]]])}
#---------------------------------------------------------------------
Theta_person = fit_gaussian_NB(Xperson, Yperson)

Theta_person

{'prior': {'vocab': array(['female', 'male'], dtype='<U6'),
  'freq': array([4, 4])},
 'likelihood': array([[[165.        ,  60.1       ,  19.        ],
         [ 92.66666667, 114.04      ,  11.33333333]],
 
        [[178.        ,  79.925     ,  28.25      ],
         [ 29.33333333,  25.47583333,   5.58333333]]])}

#### I.2.4. Gaussian Likelihood prediction

Given:
- $A$: a numerical feature
- $\mu_{Ac}$: mean of values of feature $A$ having $c$ as class
- $\sigma_{Ac}$: variance of values of feature $A$ having $c$ as class
- $Y$: the output
- $C$: the classes

Log likelihood is calculated as:
$$ \log p(A=v|Y=c) = \frac{-(v-\mu_{Ac})^2}{2 \sigma_{Ac}^2} - \log(\sqrt{2\pi \sigma_{Ac}^2})$$

In [13]:
# TODO: Gaussian Likelihood prediction
def predict_gaussian_NB1(v: str,
                         j: int,
                         Theta: object,
                         alpha: float = 0. # this is just added for compatibility
                         ) -> 'np.ndarray[C](float)':

    return - (np.power(v - Theta['likelihood'][:, 0, j], 2)) / (2 * Theta['likelihood'][:, 1, j]) - np.log(np.sqrt(2 * np.pi * Theta['likelihood'][:, 1, j]))


#=====================================================================
# UNIT TEST
#=====================================================================
# Result:
# (array([-4.93164438, -3.03443716]), array([0.00721463, 0.04810173]))
#---------------------------------------------------------------------

pp = predict_gaussian_NB1(183, 0, Theta_person)

pp, np.exp(pp)

(array([-4.93164438, -3.03443716]), array([0.00721463, 0.04810173]))

### I.4. Final prediction

Our goal is to calculate approximate log probabilities of all classes given a sample:
$$\log P(y=c_k | \overrightarrow{x} = \overrightarrow{f})  \approx \log P(y=c_k) + \sum\limits_{f_j \in \overrightarrow{f}} \log P(f_j = x_j|y=c_k)$$

This function takes:
- $X^{(i)}$ one sample with $N$ features
- $\theta$ parameters (either those of multinomial or gaussian)
- $pred_{fct}$ a function to predict one feauture (either multinomial or gaussian)
- add_prior: if True, add prior probability
- $\alpha$ smoothing factor (passing it to gaussian function will do nothing)

It must return a vector of probabilities

In [15]:
# TODO: Final prediction
def predict_NB1(Xi    : 'np.ndarray[N]',
                Theta: object,
                pred_fct: Callable,
                add_prior: bool  = True,
                alpha: float = 1.0
                ) -> 'np.ndarray[C](float)':

    yFreq = Theta['prior']['freq']
    yFreq = yFreq/sum(yFreq)
    priorLog = np.log(yFreq)

    labels = Theta['prior']['vocab']
    logSum = np.zeros([len(labels)])
    for i in range(len(Xi)):
        logSum +=pred_fct(Xi[i],i,Theta,alpha)

    return logSum+priorLog*add_prior

# Changed like in classroom
#=====================================================================
# UNIT TEST
#=====================================================================
# Result:
# (array([-3.59617006, -4.95406494]),
# array([-2.56655064, -4.51223219]),
# array([-2.85774653, -4.23617476]),
# array([-10.401093 , -22.03977023]))
#---------------------------------------------------------------------

X_t1 = np.array(['sunny', 'hot' , 'high', 'no'])
X_t2 = np.array([183., 59., 20.])

predict_NB1(X_t1, Theta_play, predict_multinomial_NB1, add_prior=True, alpha=0.0), \
predict_NB1(X_t1, Theta_play, predict_multinomial_NB1, add_prior=False, alpha=0.0), \
predict_NB1(X_t1, Theta_play, predict_multinomial_NB1, add_prior=False, alpha=1.0), \
predict_NB1(X_t2, Theta_person, predict_gaussian_NB1, add_prior=False),

(array([-3.59617006, -4.95406494]),
 array([-2.56655064, -4.51223219]),
 array([-2.85774653, -4.23617476]),
 array([-10.401093  , -22.03977023]))

### I.5. Final product

**>> Nothing to code here**


In [17]:
class NaiveBayes(object):

    def __init__(self, multinomial=True):
        if multinomial:
            self.train = fit_multinomial_NB
            self.pred = predict_multinomial_NB1
        else:
            self.train = fit_gaussian_NB
            self.pred = predict_gaussian_NB1

    def fit(self, X, Y):
        self.Theta = self.train(X, Y)

    def predict(self, X, add_prior=True, prob=False, alpha=0.):
        Y_pred = []
        for i in range(len(X)):
            Y_pred.append(predict_NB1(
                X[i,:], self.Theta, self.pred, add_prior=add_prior, alpha=alpha
                ))

        Y_pred = np.array(Y_pred)

        if prob:
            return Y_pred

        return np.choose(np.argmax(Y_pred, axis=1), self.Theta['prior']['vocab'])

# Changed like in classroom
#=====================================================================
# UNIT TEST
#=====================================================================
# Result:
# (array(['yes', 'yes', 'no'], dtype='<U3'),
# array([[-5.20912179, -4.10264337],
# [-6.30773408, -5.48893773],
# [-3.88736595, -4.67800751]]),
# array(['female', 'male'], dtype='<U6'),
# array([[-11.09424018, -22.73291741],
# [-15.27968966, -12.41764665]]))
#---------------------------------------------------------------------

multinomial_nb = NaiveBayes()
multinomial_nb.fit(Xplay, Yplay)

gaussian_nb = NaiveBayes(multinomial=False)
gaussian_nb.fit(Xperson, Yperson)

X_t1 = np.array([
    ['rainy', 'cool', 'normal', 'yes'],
    ['snowy', 'cool', 'normal', 'yes'],
    ['sunny', 'hot' , 'high', 'no']
])

X_t2 = np.array([
    [183., 59., 20.],
    [175., 65., 30.]
])


multinomial_nb.predict(X_t1, alpha=1.), \
    multinomial_nb.predict(X_t1, alpha=1., prob=True), \
    gaussian_nb.predict(X_t2), \
    gaussian_nb.predict(X_t2, prob=True)

(array(['yes', 'yes', 'no'], dtype='<U3'),
 array([[-5.20912179, -4.10264337],
        [-6.30773408, -5.48893773],
        [-3.88736595, -4.67800751]]),
 array(['female', 'male'], dtype='<U6'),
 array([[-11.09424018, -22.73291741],
        [-15.27968966, -12.41764665]]))

## II. Application and Analysis

In this section, we will test different concepts by running an experiment, formulating a hypothesis and trying to justify it.

### II.1. Prior probability

We want to test the effect of prior probability.
To do this, we trained two models:
1. With prior probability
1. Without prior probability (It considers a uniform distribution of classes)

To test whether the models have adapted well to the training dataset, we will test them on the same dataset and calculate the classification ratio.


In [ ]:
nb_withPrior     = CategoricalNB(alpha=1.0, fit_prior=True )
nb_noPrior       = CategoricalNB(alpha=1.0, fit_prior=False)

enc         = OrdinalEncoder()
Xplay_tf    = enc.fit_transform(Xplay)
nb_withPrior.fit(Xplay_tf, Yplay)
nb_noPrior.fit(Xplay_tf, Yplay)

Ypred_withPrior = nb_withPrior.predict(Xplay_tf)
Ypred_noPrior = nb_noPrior.predict(Xplay_tf)


print( 'Considring prior probability'  )
print(classification_report(Yplay, Ypred_withPrior))

print( 'No prior probability'  )
print(classification_report(Yplay, Ypred_noPrior))

Considring prior probability
              precision    recall  f1-score   support

          no       1.00      0.80      0.89         5
         yes       0.90      1.00      0.95         9

    accuracy                           0.93        14
   macro avg       0.95      0.90      0.92        14
weighted avg       0.94      0.93      0.93        14

No prior probability
              precision    recall  f1-score   support

          no       0.67      0.80      0.73         5
         yes       0.88      0.78      0.82         9

    accuracy                           0.79        14
   macro avg       0.77      0.79      0.78        14
weighted avg       0.80      0.79      0.79        14



**TODO: Analyze the results**

1. What do you notice, indicating if prior probability is useful in this case?
1. How does this probability affect the outcome?
1. When are we sure that using this probability is unnecessary?

**Answer**

1. We observe that incorporating the prior probability leads to higher overall accuracy, precision, recall, and F1-score compared to when it is not considered. This suggests that, in this case, taking the prior probability into account improves performance and is therefore beneficial.

1. This probability affects the outcome by increasing the probability of the most frequent class. We said that the output class will be the class that maximizes the following quantity: $p(y = k) \prod_{j=1}^N P(x_j | y = k), \, k \in \{1, \dots, L\}$. As a result, the most frequent class in our training set will have more chances of maximizing this quantity, making it more likely to be assigned to a new input.

1. Using the prior probability becomes unnecessary when the class distribution is uniform, meaning each class appears equally in the training data. As mentioned, the predicted class is the one that maximizes $p(y = k) \prod_{j=1}^N P(x_j | y = k)$.Since all classes have the same frequency, i.e the quantity $p(y = k)$ is equal across all classes. Maximising $p(y = k) \prod_{j=1}^N P(x_j | y = k), \, k \in \{1, \dots, L\}$ will be the same as maximizing $\prod_{j=1}^N P(x_j | y = k), \, k \in \{1, \dots, L\}$. Therefore, there's no need for prior probability

### II.2. Smoothing

We want to test the Lidstone smoothing's effect.
To do this, we trained three models:
1. alpha = 1 (Laplace smoothing)
1. alpha = 0.5
1. alpha = 0 (without smoothing)

In [ ]:
NBC_10 = CategoricalNB(alpha = 1.0 )
NBC_05 = CategoricalNB(alpha = 0.5 )
NBC_00 = CategoricalNB(alpha = 0.0 )

NBC_10.fit( Xplay_tf,   Yplay )
NBC_05.fit( Xplay_tf,   Yplay )
NBC_00.fit( Xplay_tf,   Yplay )

Y_10   = NBC_10.predict(Xplay_tf)
Y_05   = NBC_05.predict(Xplay_tf)
Y_00   = NBC_00.predict(Xplay_tf)


print(                'Alpha = 1.0'                        )
print(classification_report(Yplay, Y_10, zero_division=0.0))

print(                'Alpha = 0.5'                        )
print(classification_report(Yplay, Y_05, zero_division=0.0))

print(                'Alpha = 0.0'                        )
print(classification_report(Yplay, Y_00, zero_division=0.0))


Alpha = 1.0
              precision    recall  f1-score   support

          no       1.00      0.80      0.89         5
         yes       0.90      1.00      0.95         9

    accuracy                           0.93        14
   macro avg       0.95      0.90      0.92        14
weighted avg       0.94      0.93      0.93        14

Alpha = 0.5
              precision    recall  f1-score   support

          no       1.00      0.80      0.89         5
         yes       0.90      1.00      0.95         9

    accuracy                           0.93        14
   macro avg       0.95      0.90      0.92        14
weighted avg       0.94      0.93      0.93        14

Alpha = 0.0
              precision    recall  f1-score   support

          no       1.00      0.80      0.89         5
         yes       0.90      1.00      0.95         9

    accuracy                           0.93        14
   macro avg       0.95      0.90      0.92        14
weighted avg       0.94      0.93     

/opt/env/ml/lib/python3.10/site-packages/sklearn/naive_bayes.py:1504: RuntimeWarning: divide by zero encountered in log
  np.log(smoothed_cat_count) - np.log(smoothed_class_count.reshape(-1, 1))


**TODO: Analyze the results**

1. What do you notice, indicating if smoothing affects performance in this case?
1. Based on the past answeer, Why?
1. Why do we get a "RuntimeWarning: divide by zero" error?
1. What is the benefit of smoothing (generally; not just for this case)?

**Answer**

1. No, smoothing doesn’t make a difference here. Even with different alpha values (1, 0.5, 0), the model produces identical precision, recall, and F1-scores across classes. This consistency shows that smoothing isn’t needed for the model to reach accurate predictions, suggesting the data has no major gaps or imbalances that smoothing would typically address.

2. Smoothing is generally applied when there is a feature $x_j$ that takes a value $v$ in the test dataset that was never previously seen in the training dataset. However, since we tested the performance of three models on a test dataset which is identical to the training dataset, there isn't a case where the feature $x_j$ takes a value $v$ which wasn't already seen by the model in the training dataset, so the smoothing factor $\alpha$ wasn't used when estimating the probabilities, so the predicted output by the three models would be in this case very much identical, and thus the three models would have the same performance despite having different values for $\alpha$. also this inequality : $P(Y=\text{YES}).P(X|Y=\text{YES})>P(Y=\text{NO}).P(X|Y=\text{NO})$ will not change after adding smoothing, because smoothing adjusts both likelihoods proporitonally (smoothing does not drastically alter which probability is larger since smoothing applies the same effect to both probabilities), thus, the predicted class would still likely be "YES"

3. When predicting a new input, we said that we find the class $k$ that maximizes $P(y = k) \prod_{j=1}^N P(x_j | y = k), \, k \in \{1, \dots, L\}$, which is the same as finding the class $k$ which maximizes $\log P(y=c_k) + \sum_{j=1}^N log P(x_j | y = k), \, k \in \{1, \dots, L\} = \log P(y=c_k) + \sum_{j=1}^N \log(\#(Y = k \wedge x_j= v) + \alpha) - \log(\#(y = k) + \alpha * |V|), \, k \in \{1, \dots, L\} $ Sometimes, the quantity $\#(Y = k \wedge x_j= v)$ might be 0 (there's no instance where the feature $x_j$ takes the value $v$ and it belongs to the class $k$), and if we have set the smoothing rate to be zero ($\alpha= 0$), we could calculate a log(0), which is impossible, explains why sklearn raised an error.

4. Smoothing is typically used in Naive Bayes classification to handle the problem of zero probabilities, which can occur when a particular feature value is not present in the training data for a given class.

### II.3. Naive Bayes performance

Naive Bayes is known to generate powerful models when it comes to classifying textual documents.
We want to test this proposition using spam detection over [SMS Spam Collection Dataset](https://www.kaggle.com/uciml/sms-spam-collection-dataset) dataset.

Each message is represented using term frequency (TF), where a word is considered as a feature.
In this case, a message is represented by a vector of frequencies (how many times each word appeared in the message).
We want to compare these models:
1. Multinomial Naive Bayes (MNB)
1. Gaussian Naive Bayes (GNB)
1. Logistic Regression (LR)

In [ ]:
# reading the dataset
messages = pd.read_csv('data/spam.csv', encoding='latin-1')
# renaming features: text and class
messages = messages.rename(columns={'v1': 'class', 'v2': 'text'})
# keeping only these two features
messages = messages.filter(['text', 'class'])

messages.head()

,text,class
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham


In [ ]:
models = [
    MultinomialNB(),
    GaussianNB(),
    LogisticRegression(solver='lbfgs'),
    #solver=sag is slower; so I chose the fastest
]

algos = [
    'Multinomial Naive Bayes (MNB)',
    'Gaussian Naive Bayes  (GNB)',
    'Logistic Regression (LR)',
]

perf = {
    'train_time': [],
    'test_time' : [],
    'recall'    : [],
    'precision' : []
}


msg_train, msg_test, Y_train, Y_test = train_test_split(messages['text'] ,
                                                        messages['class'],
                                                        test_size    = 0.2,
                                                        random_state = 0  )

count_vectorizer = CountVectorizer()
X_train          = count_vectorizer.fit_transform(msg_train).toarray()
X_test           = count_vectorizer.transform    (msg_test ).toarray()


for model in models:
    # ==================================
    # TRAIN
    # ==================================
    start_time = timeit.default_timer()
    model.fit(X_train, Y_train)
    perf['train_time'].append(timeit.default_timer() - start_time)

    # ==================================
    # TEST
    # ==================================
    start_time = timeit.default_timer()
    Y_pred     = model.predict(X_test)
    perf['test_time'].append(timeit.default_timer() - start_time)

    # ==================================
    # PERFORMANCE
    # ==================================
    # In here, we are interrested in "spam" class which is our positive class
    perf['precision'].append(precision_score(Y_test, Y_pred, pos_label='spam'))
    perf['recall'   ].append(recall_score   (Y_test, Y_pred, pos_label='spam'))


pd.DataFrame({
    'Algorithm' : algos,
    'Train time': perf['train_time'],
    'Test time' : perf['test_time'],
    'Precision' : perf['precision'],
    'Recall'    : perf['recall']
})

,Algorithm,Train time,Test time,Precision,Recall
0,Multinomial Naive Bayes (MNB),0.288778,0.021871,0.987179,0.927711
1,Gaussian Naive Bayes (GNB),0.420813,0.095978,0.616667,0.891566
2,Logistic Regression (LR),0.551115,0.022939,0.986111,0.855422


**TODO: Analyze the results**

1. What do you notice about training time? (order the algorithms)
1. Why did we get these results based on the algorithms? (discuss each algorithm with respect to training time)
1. What do you notice about the testing time? (order the algorithms)
1. Why did we get these results based on the algorithms? (discuss each algorithm with respect to testing time)
1. Why is the Gaussian model less efficient than the multinomial based on the nature of the two algorithms?
1. Why is the Gaussian model less efficient than the multinomial based on the nature of the problem/data?
1. How Multinomial NB's implementation affect the training/test time? (store statistics vs. store probabilities)
1. Which one is more adequate for updating the model with new data? explain.

**Answer**

1. The fastest model in terms of training time is: MNB (Multinominal Naive Bayes), followed by GNB (Gaussian Naive Bayes) in second place, and finally LR (Logistic Regression) has the slowest training time (0.551115 seconds).

2. Naive Bayes algorithms (both Multinominal and Gaussian) store statistical parameters that are later used to predict the output class of a new entry. For example, they store the number of samples for each class. In the case of Multinominal Naive Bayes, for each class $k$ and feature $x_j$, the model calculates the number of instances belonging to the class $k$ that have the value $v$ for the feature $x_j$. For Gaussian Naive Bayes, we calculate $\mu_{kj}$ (the mean of $x_j$’s values having $k$ as class) and $\sigma^2_{kj}$ (the unbiased variance of $x_j$’s values having $k$ as class). In contrast, a logistic regression model needs to update its parameters in an iterative manner using methods such as gradient descent, which is more computantionally expensive. This explains why Naive Bayes algorithms have a faster training time compared to Logistic Regression.

3. The fastest model in terms of testing time is: MNB (Multinominal Naive Bayes), followed by LR (Logistic Regression) in second place, and finally GNB (Gaussian Naive Bayes) has the slowest testing time (0.095978 seconds).

4. MNB had the fastest testing time because it simply relies on word frequencies to predict the class. Logistic regression has a slightly slower testing time than MNB because it involves calculating weighted sums of all features and then applying a sigmoid function to predict the class. Although still efficient, it requires more calculations than MNB. Finally, GNB has the slowest testing time because it calculates likelihood probabilities using the normal distribution, which involves more complex calculations, resulting in a much slower testing time.

5. GNB assumes a Gaussian (Normal) distribution for each feature, which doesn’t align well with discrete features in text classification as it has to estimate probabilities for a distribution that doesn’t naturally fit. MNB, on the other hand, is specifically designed for discrete data, so it operates more efficiently in this context.

6. The Gaussian model is less efficient because the features represent word counts within a text, which are discrete rather than continuous. Gaussian Naive Bayes is more suitable for continuous features. Multinomial Naive Bayes, however, is designed for handling discrete data, making it a better performing model for this kind of problem.

7. Store statistics: Training is faster because the model only store counts for the different classes. More precisely, for each class $k$, it records the number of instances belonging to that class. Additionally, for each class $k$, for each feature $x_j$, for each value $v$ from the possible values of $x_j$, we count the number of instances that have the value $v$ for the feature $x_j$ and belong to the class $k$. However it's slower in testing because it needs to calculate the probabilities (prior + likelihood) everytime for each sample in the test data set to make a prediction.

 Store Probabilities: If the model stores precalculated probabilities directly, the test time can be faster, as it avoids the need to recalculate probabilities during testing. However, training might be slightly slower as the model needs to compute and store the probabilities.


8. The Multinominal Naive Bayes approach which stores statistics is more efficient for updating the model with new data since it only requires adjusting counts according to the new samples, which can be done in an incremental way, without having to re-do the entire calculations. In contrast, the Multinominal Naive Bayes approach which directly stores probabilities is less suitable in this case because it requires re-calculating probabilities from scratch, which is a computationally intensive task.

In [ ]:
print("  _____    __                                              _               ")
print(" |_   _|  / _|                                            | |              ")
print("   | |   | |_     _   _    ___    _   _      __ _    ___  | |_             ")
print("   | |   |  _|   | | | |  / _ \  | | | |    / _` |  / _ \ | __|            ")
print("  _| |_  | |     | |_| | | (_) | | |_| |   | (_| | |  __/ | |_             ")
print(" |_____| |_|      \__, |  \___/   \__,_|    \__, |  \___|  \__|            ")
print("                   __/ |                     __/ |                         ")
print("                  |___/                     |___/                          ")
print("  _     _       _            __                                            ")
print(" | |   | |     (_)          / _|                 _                         ")
print(" | |_  | |__    _   ___    | |_    __ _   _ __  (_)                        ")
print(" | __| | '_ \  | | / __|   |  _|  / _` | | '__|                            ")
print(" | |_  | | | | | | \__ \   | |   | (_| | | |     _                         ")
print("  \__| |_| |_| |_| |___/   |_|    \__,_| |_|    ( )                        ")
print("                                                |/                         ")
print("                                                                           ")
print("                                                                           ")
print("                                                                           ")
print("  _   _    ___    _   _      __ _   _ __    ___                            ")
print(" | | | |  / _ \  | | | |    / _` | | '__|  / _ \                           ")
print(" | |_| | | (_) | | |_| |   | (_| | | |    |  __/                           ")
print("  \__, |  \___/   \__,_|    \__,_| |_|     \___|                           ")
print("   __/ |                                                                   ")
print("  |___/                                                                    ")
print("                    _                                                __    ")
print("                   | |                                            _  \ \   ")
print("  _ __     ___     | |__    _   _   _ __ ___     __ _   _ __     (_)  | |  ")
print(" | '_ \   / _ \    | '_ \  | | | | | '_ ` _ \   / _` | | '_ \         | |  ")
print(" | | | | | (_) |   | | | | | |_| | | | | | | | | (_| | | | | |    _   | |  ")
print(" |_| |_|  \___/    |_| |_|  \__,_| |_| |_| |_|  \__,_| |_| |_|   (_)  | |  ")
print("                                                                     /_/   ")
print("                                                                           ")

  _____    __                                              _               
 |_   _|  / _|                                            | |              
   | |   | |_     _   _    ___    _   _      __ _    ___  | |_             
   | |   |  _|   | | | |  / _ \  | | | |    / _` |  / _ \ | __|            
  _| |_  | |     | |_| | | (_) | | |_| |   | (_| | |  __/ | |_             
 |_____| |_|      \__, |  \___/   \__,_|    \__, |  \___|  \__|            
                   __/ |                     __/ |                         
                  |___/                     |___/                          
  _     _       _            __                                            
 | |   | |     (_)          / _|                 _                         
 | |_  | |__    _   ___    | |_    __ _   _ __  (_)                        
 | __| | '_ \  | | / __|   |  _|  / _` | | '__|                            
 | |_  | | | | | | \__ \   | |   | (_| | | |     _                         
  \__| |_| |